In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [7]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset[['Purchased']]

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid={'solver':['newton-cg','lbfgs','liblinear','saga'],'penalty':['l2']}
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
grid_pred=grid.predict(x_test)

In [15]:
from sklearn.metrics import confusion_matrix,classification_report,f1_score,roc_auc_score
cm=confusion_matrix(y_test,grid_pred)
clf_report=classification_report(y_test,grid_pred)
f1_macro=f1_score(y_test,grid_pred,average='weighted')
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9507803121248499

In [16]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[79  6]
 [10 39]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [20]:
print("The f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for the best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.8794289739855382


In [21]:
table=pd.DataFrame.from_dict(re)

In [22]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011883,0.000983,0.008212,0.001762,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
1,0.011442,0.002404,0.008763,0.000848,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
2,0.012408,0.002003,0.008831,0.000875,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
3,0.003468,0.003277,0.007135,0.000698,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2


In [23]:
age=float(input("Age:"))
salary=float(input("Salary:"))
sex=int(input("Male 0 or 1:"))

Age: 45
Salary: 35000
Male 0 or 1: 0


In [25]:
future_prediction=grid.predict([[age,salary,sex]])

In [26]:
print("The Future Prediction {}:".format(future_prediction))

The Future Prediction [1]:
